In [ ]:
import requests
import random
import re
import os
from bs4 import BeautifulSoup
from pymongo import MongoClient
import datetime
import time

list_page_1 = []
list_page_2 = []
list_page_3 = []

#加入page,使其能够把所有page的都搞下来
#每个都放到不同的文件夹里面
#文件夹存在判断
#去重

class mzitu:
    def __init__(self):
        client = MongoClient() ##与MongDB建立连接（这是默认连接本地MongDB数据库）
        db = client['meinvxiezhenji'] ## 选择一个数据库
        self.meizitu_collection = db['meizitu'] ##在meizixiezhenji这个数据库中，选择一个集合
        self.title = '' ##用来保存页面主题
        self.url = '' ##用来保存页面地址
        self.img_urls = [] ##初始化一个 列表 用来保存图片地址
    
    def html(self,page_url):
        html = Download.get(page_url,20)
        html.encoding = 'gbk'
        page_html_Soup = BeautifulSoup(html.text,'lxml')
        max_span = page_html_Soup.find('div',{'id':'wp_page_numbers'}).find_all('a')[-3].get_text()
        for page in range(1, int(max_span) + 1):
            page_url = 'http://www.meizitu.com/a/xiaoqingxin_6_' + str(page) + '.html'
            list_page_1.append(page_url)
        print(list_page_1)
        
        for page in list_page_1:
            html = Download.get(page,20)
            html.encoding = 'gbk'
            html_Soup = BeautifulSoup(html.text,'lxml')
            li_list = html_Soup.find('ul',class_ = 'wp-list').find_all('a')
            li_list = li_list[1::2]
            list_page_3.extend(li_list)            
        print(list_page_3)
        self.page(list_page_3)
        
    def page(self,list_page_3): 
        for a in list_page_3:
            title = a.get_text()
            self.title = title
            print(u'开始保存',title)
            path = title
            href = a['href']
            self.url = href
            print(title,href)
            if self.meizitu_collection.find_one({'主题页面':href}):
                print(u'这个页面已经爬取过了')
            else:
                self.img(href,path)
            
    def img(self,href,path):
        self.mkdir(path) ##调用mkdir函数创建文件夹！这儿path代表的是标题title哦！！！！！不要糊涂了哦！
        img_html = Download.get(href,20)
        img_html.encoding = 'gbk'
        img_Soup = BeautifulSoup(img_html.text,'lxml')
        img_list = img_Soup.find('div',{'id':'maincontent'}).find_all('img')
        k = img_Soup.find('div',{'id':'maincontent'}).find('img')['src']
        self.img_urls.append(k)
        print(img_list)
        for img in img_list:
            name = img['alt'][0:]
            url_img = img['src']
            im = Download.get(url_img,20) 
            f = open(name+'.jpg', 'ab')##写入多媒体文件必须要 b 这个参数！！必须要！！
            f.write(im.content) ##多媒体文件要是用conctent哦！
        f.close()
        post = {
            '标题':self.title,
            '主题页面':self.url,
            '图片地址':self.img_urls,
            '获取时间':datetime.datetime.now()
        }
        self.meizitu_collection.save(post)
        print(u'插入数据库成功')
                    
    def request(self,url):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1'}
        content = requests.get(url,headers = headers)
        return content
       
    def mkdir(self,path):
        path = path.strip()
        isExists = os.path.exists(os.path.join('C:\Meizitu',path))
        if not isExists:
            os.makedirs(os.path.join("C://Meizitu",path))
            a = str("C://Meizitu/"+ path)
            os.chdir(a)
            return True
        else:
            print(u'名字叫做',path,u'的文件夹已经存在了')
            return False
        
class download:
    def __init__(self):        
        self.iplist = []
        html = requests.get("http://haoip.cc/tiqu.htm")
        iplistn = re.findall(r'r/>(.*?)<b',html.text,re.S)
        for ip in iplistn:
                 i = re.sub('\n','',ip)
        self.iplist.append(i.strip())
        self.user_agent_list = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
 "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
 "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
 "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
 "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
 "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
 "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
 "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
 "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
 "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"]
               
    def get(self, url, timeout, proxy = None , num_retries = 4):
        UA = random.choice(self.user_agent_list)
        headers = {'User-Agent':UA}
        if proxy == None:
            try:                
                return requests.get(url,headers = headers,timeout = timeout)
            except:
                if num_retries > 0:
                    time.sleep(10)
                    print(u'获取网页出错，10S后将获取倒数第：', num_retries, u'次')
                    return self.get(url,timeout,num_retries-1)
                else:
                    print(u'开始使用代理')
                    time.sleep(10)
                    IP = ''.join(str(random.choice(self.iplist)).strip())
                    proxy = {'http':IP}
                    return self.get(url,timeout,proxy,)
        else:
            IP = ''.join(str(random.choice(self.iplist)).strip())##当代理不为空
            proxy = {'http':IP}
            response = requests.get(url,headers = headers, proxies = proxy)
            return response 
                 
Download = download()
page_url_1 = 'http://www.meizitu.com/a/xiaoqingxin_6_1.html'
Mzitu = mzitu()
Mzitu.html(page_url_1)

['http://www.meizitu.com/a/xiaoqingxin_6_1.html', 'http://www.meizitu.com/a/xiaoqingxin_6_2.html', 'http://www.meizitu.com/a/xiaoqingxin_6_3.html', 'http://www.meizitu.com/a/xiaoqingxin_6_4.html', 'http://www.meizitu.com/a/xiaoqingxin_6_5.html', 'http://www.meizitu.com/a/xiaoqingxin_6_6.html']
[<a href="http://www.meizitu.com/a/5460.html" target="_blank"><b>怪蜀黍的最爱，小清新风</b></a>, <a href="http://www.meizitu.com/a/5323.html" target="_blank">热爱运动的妹纸，趁着放晴出来打棒球</a>, <a href="http://www.meizitu.com/a/5296.html" target="_blank">小清新妹子向我们展示完美笑容</a>, <a href="http://www.meizitu.com/a/5234.html" target="_blank"><b>可爱妹子与狗狗野餐记</b></a>, <a href="http://www.meizitu.com/a/5224.html" target="_blank">隔户杨柳弱袅袅，恰似十五女儿腰</a>, <a href="http://www.meizitu.com/a/5181.html" target="_blank"><b>美好的东西总是一言难尽</b></a>, <a href="http://www.meizitu.com/a/5180.html" target="_blank">日系妹子在咖啡店的简单生活</a>, <a href="http://www.meizitu.com/a/5166.html" target="_blank"><b>甜美长发少女的十八歲</b></a>, <a href="http://www.meizitu.com/a/5162.